In [34]:
import json, sys, random, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, RMSprop, Adam


In [23]:
X = []
y = []
encoder = OneHotEncoder()
encoder.fit([[0], [1]])


OneHotEncoder()

In [24]:
# Load images from the "yes" directory
for r, d, f in os.walk(r'/Users/rahulkushwah/Documents/JUPYTER/archive (1)/yes'):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((240, 240))
            img = np.array(img)
            if img.shape == (240, 240, 3):
                X.append(img)
                y.append(0)  # 0 for "yes"


In [25]:
# Load images from the "no" directory
for r, d, f in os.walk(r'/Users/rahulkushwah/Documents/JUPYTER/archive (1)/no'):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((240, 240))
            img = np.array(img)
            if img.shape == (240, 240, 3):
                X.append(img)
                y.append(1)  # 1 for "no"


In [26]:
X = np.array(X)
y = np.array(y)


In [27]:
# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


In [28]:
# Define the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(240, 240, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])


In [29]:
# Train the model
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=2, verbose=2, mode='auto')
mc = ModelCheckpoint(filepath="../kaggle/working/best_model.h5", monitor='val_accuracy', save_best_only=True)
history = model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=5, callbacks=[mc, es])


Epoch 1/5
55/55 [==============================] - ETA: 0s - loss: 24.9461 - accuracy: 0.7266

/Users/rahulkushwah/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


55/55 [==============================] - 72s 1s/step - loss: 24.9461 - accuracy: 0.7266 - val_loss: 0.3026 - val_accuracy: 0.8547
Epoch 2/5
55/55 [==============================] - 77s 1s/step - loss: 0.1980 - accuracy: 0.9239 - val_loss: 0.1943 - val_accuracy: 0.9273
Epoch 3/5
55/55 [==============================] - 81s 1s/step - loss: 0.1178 - accuracy: 0.9579 - val_loss: 0.1655 - val_accuracy: 0.9689
Epoch 4/5
55/55 [==============================] - 79s 1s/step - loss: 0.0127 - accuracy: 0.9954 - val_loss: 0.1662 - val_accuracy: 0.9654
Epoch 5/5
55/55 [==============================] - 82s 1s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.1721 - val_accuracy: 0.9706
Epoch 5: early stopping


In [30]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


19/19 [==============================] - 6s 290ms/step - loss: 0.1462 - accuracy: 0.9672
Test Loss: 0.14616666734218597
Test Accuracy: 0.9671847820281982


In [33]:
# Convert history object to DataFrame
history_df = pd.DataFrame(history.history)
print(history_df)

        loss  accuracy  val_loss  val_accuracy
0  24.946064  0.726644  0.302573      0.854671
1   0.198047  0.923875  0.194299      0.927336
2   0.117850  0.957901  0.165540      0.968858
3   0.012726  0.995386  0.166189      0.965398
4   0.001892  1.000000  0.172067      0.970588
